In [ ]:
from pynq import Overlay
import os
import sys

HOP_DIR=os.path.abspath("../")
sys.path.insert(0, HOP_DIR)
import pushpush

OVERLAY_DIR=f'{HOP_DIR}/overlays/a_add/'

In [ ]:
ol = Overlay(OVERLAY_DIR + "a_add.bit")
ol.ip_dict

In [ ]:
import importlib as il
if 'context' in locals():
    del context
    il.reload(pushpush)
    pushpush.Context.reloadModules()
context = pushpush.Context(ol)

In [ ]:
def py_func1():
    print("Hello from func1")
    return 58

def py_func2():
    print("Hello from func2")    
    return 1000

def py_func3():
    print("Hello from func3")
    return 100

a_py = context.register(py_func1, "() -> b32")
b_py = context.register(py_func2, "() -> b32")
c_py = context.register(py_func3, "() -> b32")
d_var = context.register(1000, 'b32')

In [ ]:
context.print_all_objects()

In [ ]:
a = context.functions['hardware']['a']
add = context.functions['hardware']['add']

In [ ]:
x = context.register(add(a_py, d_var), 'b32')
print(x)
y = context.register(add(a, b_py), 'b32')
print(str(y))
z = add(x, y)
print(z)